<h4>Добавьте туда файл conftest.py из предыдущего модуля. <br>Убедитесь дополнительно, что там есть параметр для задания языка интерфейса, по умолчанию равный "en".</h4>

In [8]:
%%writefile conftest.py

import pytest
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def pytest_addoption(parser):
    parser.addoption('--browser_name', action='store', default="chrome",
                     help="Choose browser: chrome or firefox")
    parser.addoption('--language', action='store', default="en",
                     help="Choose language")
    
@pytest.fixture(scope="function")
def language(request):
    language = request.config.getoption("language")
    return language 

@pytest.fixture(scope="function")
def browser(request,language):
    
    browser_name = request.config.getoption("browser_name")
    browser = None
    
    if browser_name == "chrome":
        
        print("\nstart chrome browser for test..")
        options = Options()
        options.add_experimental_option('prefs', {'intl.accept_languages': language})
        browser = webdriver.Chrome(options=options)
        
    elif browser_name == "firefox":
        
        print("\nstart firefox browser for test..")
        fp = webdriver.FirefoxProfile()
        fp.set_preference("intl.accept_languages", language)
        browser = webdriver.Firefox(firefox_profile=fp)

    else:
        
        raise pytest.UsageError("--browser_name should be chrome or firefox")
        
    yield browser
    print("\nquit browser..")
    browser.quit()

    

Overwriting conftest.py


#### Создайте файл test_main_page.py и добавьте в него тест из предыдущего модуля:  

In [9]:
%%writefile test_main_page.py

def test_guest_can_go_to_login_page(browser):
    link = "http://selenium1py.pythonanywhere.com/"
    browser.get(link)
    login_link = browser.find_element_by_css_selector("#login_link")
    login_link.click()

Overwriting test_main_page.py


In [11]:
!pytest -v --tb=line --language=en test_main_page.py

============================= test session starts ==============================
platform darwin -- Python 3.10.0, pytest-6.2.5, py-1.11.0, pluggy-0.13.1 -- /Users/silinanata/environments/selenium_env/bin/python3
cachedir: .pytest_cache
rootdir: /Users/silinanata/selenium_project/first-selenium-project
plugins: rerunfailures-10.2
collected 1 item                                                               

test_main_page.py::test_guest_can_go_to_login_page PASSED                [100%]

============================== 1 passed in 8.56s ===============================


In [ ]:
%writefile pages/base_page.py

class BasePage():
    
    def __init__(self, browser, url):
        self.browser = browser
        self.url = url
    
    def open(self):
        self.browser.get()